In [1]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
path = r'./datasets/U.S._Chronic_Disease_Indicators__CDI_.csv'

In [3]:
df = pd.read_csv(path)
df.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueTypeID,...,TopicID,QuestionID,ResponseID,LocationID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
0,2013,2013,CA,California,YRBSS,Alcohol,Alcohol use among youth,NaN,%,CrdPrev,...,ALC,ALC1_1,NaN,6,OVERALL,OVR,NaN,NaN,NaN,NaN
1,2013,2013,CO,Colorado,YRBSS,Alcohol,Alcohol use among youth,NaN,%,CrdPrev,...,ALC,ALC1_1,NaN,8,OVERALL,OVR,NaN,NaN,NaN,NaN
2,2013,2013,CT,Connecticut,YRBSS,Alcohol,Alcohol use among youth,NaN,%,CrdPrev,...,ALC,ALC1_1,NaN,9,OVERALL,OVR,NaN,NaN,NaN,NaN
3,2013,2013,DC,District of Columbia,YRBSS,Alcohol,Alcohol use among youth,NaN,%,CrdPrev,...,ALC,ALC1_1,NaN,11,OVERALL,OVR,NaN,NaN,NaN,NaN
4,2013,2013,DE,Delaware,YRBSS,Alcohol,Alcohol use among youth,NaN,%,CrdPrev,...,ALC,ALC1_1,NaN,10,OVERALL,OVR,NaN,NaN,NaN,NaN


In [4]:
(df.count()/len(df)) * 100      # percentage of not-null entries in each column of DataFrame 

YearStart                    100.000000
YearEnd                      100.000000
LocationAbbr                 100.000000
LocationDesc                 100.000000
DataSource                   100.000000
Topic                        100.000000
Question                     100.000000
Response                      32.717966
DataValueUnit                 86.240182
DataValueTypeID              100.000000
DataValueType                100.000000
DataValue                     79.598338
DataValueAlt                  69.757649
DataValueFootnoteSymbol       53.960523
DatavalueFootnote             53.883620
LowConfidenceLimit            59.416459
HighConfidenceLimit           59.416459
StratificationCategory1      100.000000
Stratification1              100.000000
StratificationCategory2       32.717966
Stratification2               32.717966
StratificationCategory3       32.717966
Stratification3               32.717966
GeoLocation                   99.395699
TopicID                      100.000000


In [5]:
df.dropna(how='all',subset=['DataValue','DataValueAlt'],inplace=True) # Dropping rows where DataValue and DataValueAlt has null values
df.dropna(how='all',subset=['GeoLocation'],inplace=True)  # Dropping rows where Geolocation has null values.
df.dropna(how = 'all',subset=['LowConfidenceLimit','HighConfidenceLimit'],inplace = True) #Dropping rows where LowConfidenceLimit and HighConfidenceLimit has null values.

In [6]:
# These features have most of thier data as null values...
features_with_maximum_null_values = [
    'Response', 'StratificationCategory2', 'Stratification2',
    'StratificationCategory3', 'Stratification3', 'ResponseID',
    'StratificationCategoryID2', 'StratificationID2',
    'StratificationCategoryID3', 'StratificationID3'
]

# These features encode same data as other features...
features_encoding_similar_info = [
    'LocationAbbr','LocationDesc',
    'Topic','DataValueType',
    'Stratification1','StratificationCategory1',
    'Question','DataValueAlt'
]
# These features does not have any info about the target...
features_with_no_info= [
    'DataValueFootnoteSymbol','DatavalueFootnote'
]


In [7]:
# Dropped the above mentioned Columns...
df.drop(columns=features_encoding_similar_info,inplace=True)
df.drop(columns=features_with_maximum_null_values,inplace=True)
df.drop(columns=features_with_no_info,inplace=True)

In [8]:
df.head()

,YearStart,YearEnd,DataSource,DataValueUnit,DataValueTypeID,DataValue,LowConfidenceLimit,HighConfidenceLimit,GeoLocation,TopicID,QuestionID,LocationID,StratificationCategoryID1,StratificationID1
2,2013,2013,YRBSS,%,CrdPrev,36.7,32.7,41.0,"(41.56266102000046, -72.64984095199964)",ALC,ALC1_1,9,OVERALL,OVR
3,2013,2013,YRBSS,%,CrdPrev,31.4,30.2,32.5,"(38.907192, -77.036871)",ALC,ALC1_1,11,OVERALL,OVR
4,2013,2013,YRBSS,%,CrdPrev,36.3,33.7,39.0,"(39.008830667000495, -75.57774116799965)",ALC,ALC1_1,10,OVERALL,OVR
5,2013,2013,YRBSS,%,CrdPrev,34.8,33.1,36.6,"(28.932040377000476, -81.92896053899966)",ALC,ALC1_1,12,OVERALL,OVR
6,2013,2013,YRBSS,%,CrdPrev,27.9,23.8,32.3,"(32.83968109300048, -83.62758034599966)",ALC,ALC1_1,13,OVERALL,OVR


In [9]:
# Making two features Geo_lat and Geo_lon out of the content of the feature Geolocation ...
df['Geo_lat'] = df.GeoLocation.map(lambda x : x.split(',')[0][1:])
df['Geo_lon'] = df.GeoLocation.map(lambda x : x.split(',')[1][:-1])

In [10]:
df.head()

,YearStart,YearEnd,DataSource,DataValueUnit,DataValueTypeID,DataValue,LowConfidenceLimit,HighConfidenceLimit,GeoLocation,TopicID,QuestionID,LocationID,StratificationCategoryID1,StratificationID1,Geo_lat,Geo_lon
2,2013,2013,YRBSS,%,CrdPrev,36.7,32.7,41.0,"(41.56266102000046, -72.64984095199964)",ALC,ALC1_1,9,OVERALL,OVR,41.56266102000046,-72.64984095199964
3,2013,2013,YRBSS,%,CrdPrev,31.4,30.2,32.5,"(38.907192, -77.036871)",ALC,ALC1_1,11,OVERALL,OVR,38.907192,-77.036871
4,2013,2013,YRBSS,%,CrdPrev,36.3,33.7,39.0,"(39.008830667000495, -75.57774116799965)",ALC,ALC1_1,10,OVERALL,OVR,39.008830667000495,-75.57774116799965
5,2013,2013,YRBSS,%,CrdPrev,34.8,33.1,36.6,"(28.932040377000476, -81.92896053899966)",ALC,ALC1_1,12,OVERALL,OVR,28.932040377000476,-81.92896053899966
6,2013,2013,YRBSS,%,CrdPrev,27.9,23.8,32.3,"(32.83968109300048, -83.62758034599966)",ALC,ALC1_1,13,OVERALL,OVR,32.83968109300048,-83.62758034599966


In [11]:
df.dtypes

YearStart                      int64
YearEnd                        int64
DataSource                    object
DataValueUnit                 object
DataValueTypeID               object
DataValue                     object
LowConfidenceLimit           float64
HighConfidenceLimit          float64
GeoLocation                   object
TopicID                       object
QuestionID                    object
LocationID                    object
StratificationCategoryID1     object
StratificationID1             object
Geo_lat                       object
Geo_lon                       object
dtype: object

In [12]:
df['DataValue'] = df.DataValue.astype(np.float64)
df['Geo_lat'] = df.Geo_lat.astype(np.float64)
df['Geo_lon'] = df.Geo_lon.astype(np.float64)

In [13]:
df.head()

,YearStart,YearEnd,DataSource,DataValueUnit,DataValueTypeID,DataValue,LowConfidenceLimit,HighConfidenceLimit,GeoLocation,TopicID,QuestionID,LocationID,StratificationCategoryID1,StratificationID1,Geo_lat,Geo_lon
2,2013,2013,YRBSS,%,CrdPrev,36.7,32.7,41.0,"(41.56266102000046, -72.64984095199964)",ALC,ALC1_1,9,OVERALL,OVR,41.562661,-72.649841
3,2013,2013,YRBSS,%,CrdPrev,31.4,30.2,32.5,"(38.907192, -77.036871)",ALC,ALC1_1,11,OVERALL,OVR,38.907192,-77.036871
4,2013,2013,YRBSS,%,CrdPrev,36.3,33.7,39.0,"(39.008830667000495, -75.57774116799965)",ALC,ALC1_1,10,OVERALL,OVR,39.008831,-75.577741
5,2013,2013,YRBSS,%,CrdPrev,34.8,33.1,36.6,"(28.932040377000476, -81.92896053899966)",ALC,ALC1_1,12,OVERALL,OVR,28.932040,-81.928961
6,2013,2013,YRBSS,%,CrdPrev,27.9,23.8,32.3,"(32.83968109300048, -83.62758034599966)",ALC,ALC1_1,13,OVERALL,OVR,32.839681,-83.627580


In [14]:
df.drop(columns='GeoLocation',inplace=True)

In [15]:
#df.to_csv('./datasets/US-Chronic-dataset.csv')